In [10]:
import torch
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)} is available.")
else:
    print("No GPU available. Training will run on CPU.")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

GPU: NVIDIA GeForce RTX 2070 SUPER is available.


In [11]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline

# TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T
# TinyLlama/TinyLlama-1.1B-Chat-v1.0
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"


tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)


pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=device)


In [12]:
# prompt 樣板設定測試
messages = [
    {   "role": "user", 
        "content": "Do you know what is the Monoploy?"
    }
]
# 設定 prompt 樣板，方便觀察輸出


In [13]:
outputs = pipe(messages, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
outputs = outputs[0]["generated_text"]
assistant_reply = next((msg["content"] for msg in outputs if msg["role"] == "assistant"), "")
print(assistant_reply)

Yes, the Monoploy is a board game that has been around since the early 1980s. It is a strategy game in which players try to accumulate a certain number of points by placing and removing cards from a deck, and then scoring points based on their cards. The game is played by two to four players, and it is a popular choice for family gatherings and social games.


In [14]:
# 設定 prompt 樣板，方便觀察輸出
chat_history = pipe.tokenizer.apply_chat_template(outputs, tokenize=False, add_generation_prompt=False)
print(chat_history)

<|user|>
Do you know what is the Monoploy?</s>
<|assistant|>
Yes, the Monoploy is a board game that has been around since the early 1980s. It is a strategy game in which players try to accumulate a certain number of points by placing and removing cards from a deck, and then scoring points based on their cards. The game is played by two to four players, and it is a popular choice for family gatherings and social games.</s>



RAG

![image.png](MarkDownImage\RAG.png)

In [15]:
import os
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores  import Chroma
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate

model_kwargs = {'device': 'cuda'}
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedding = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    model_kwargs=model_kwargs
    )
    

pdf_folder = "documents/monopoly"

pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith('.pdf')]

documents = []

for pdf_file in pdf_files:
    document = os.path.join(pdf_folder, pdf_file)
    pdf_loader = PyPDFLoader(document)
    documents.extend(pdf_loader.load())


In [16]:
vectordb= Chroma.from_documents(
    documents, 
    embedding,
    persist_directory="./knowledge-base"
)

In [17]:
def retrieve_documents(query, num_results=1):
    query_embedding = embedding.embed_query(query)
    results = vectordb.similarity_search_by_vector(query_embedding, k=num_results)
    retrieved_docs = [result.page_content for result in results]
    return retrieved_docs

def generate_answer(query, retrieved_docs):
    
    retrieved_docs_text = "\n".join(retrieved_docs)
    
    content = f"""
You're a professional assistant
Given the following documents, answer the user's question as accurately as possible:

Documents:
{retrieved_docs_text}

Question:
{query}
"""
    messages = [
        {   "role": "system", 
            "content": content,
        }
    ]
    result = pipe(messages, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
    outputs = result[0]['generated_text']
    
    assistant_reply = next((msg["content"] for msg in outputs if msg["role"] == "assistant"), "")
    
    return assistant_reply, retrieved_docs_text


def process_query(query, num_results=1):

    retrieved_docs = retrieve_documents(query, num_results=num_results)
    answer, retrieved_docs_text = generate_answer(query, retrieved_docs)
    
    return answer, retrieved_docs_text


In [18]:
query = "Do you know what is Monopoly?"
answer, retrieved_docs_text = process_query(query)
print(answer)

Yes, Monopoly is a popular board game that originated in the United States during the Great Depression. The game involves players buying and trading properties and collecting rent, and players must balance their spending with their savings to become wealthy players.
